Importando bibliotecas

In [10]:
from plyer import notification 
import requests
import pandas as pd
import sqlite3
from datetime import datetime


Criação de Alerta


In [13]:
notification.notify(
     title='Título da notificação',
     message='Mensagem da notificação',
     app_name='Nome do aplicativo',
     timeout=10 )

from datetime import datetime
def alerta(nivel, base, etapa):
    '''
        Alerta de falha de carregamento de base de dados
    '''
    now = str(datetime.now())

    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

alerta(nivel = 2,
        base = "ESTADOS", 
        etapa = "EXTRACAO")




In [4]:
"""
    Função que consome a API dos estados do Brasil (IBGE) e retorna um DataFrame com
    informações sobre os estados brasileiros.

    O DataFrame resultante contém as colunas:
    - UF: Sigla do estado
    - Estado: Nome completo do estado
    - Regiao_Nome: Nome da região do estado
    - Regiao_Codigo: Código da região do estado

    Retorna:
        pd.DataFrame: DataFrame com informações dos estados e suas regiões.
        None: Se houver erro na requisição à API.
    """

def estados_ibge():
    url = "https://brasilapi.com.br/api/ibge/uf/v1"
    response = requests.get(url)


    if response.status_code == 200:
        data_json = response.json()
        estados_ibge_df = pd.DataFrame(data_json)
        
        ## Tratamento dos dados
        # Renomeando colunas 
        estados_ibge_df = estados_ibge_df.rename(columns={
            'sigla': 'UF', 
            'nome': 'Estado',
            'regiao': 'Regiao'
        })
        
        # Expandindo os dados da coluna 'Regiao'
        estados_ibge_df['Regiao_Nome'] = estados_ibge_df['Regiao'].apply(lambda x: x['nome'])
        estados_ibge_df['Regiao_Codigo'] = estados_ibge_df['Regiao'].apply(lambda x: x['id'])
        
        # Excluindo a coluna 'Regiao'
        estados_ibge_df = estados_ibge_df.drop(columns=['Regiao'])
        
        # Excluindo colunas desnecessárias
        if 'id' in estados_ibge_df.columns:
            estados_ibge_df = estados_ibge_df.drop(columns=['id'])
        
        # Alterando tipo de dados das colunas
        estados_ibge_df['UF'] = estados_ibge_df['UF'].astype(str)
        estados_ibge_df['Estado'] = estados_ibge_df['Estado'].astype(str)

        return estados_ibge_df
    else:
        print("Erro ao acessar a API de Estados")
        return None

estados_ibge().head(10)





,UF,Estado,Regiao_Nome,Regiao_Codigo
0,RO,Rondônia,Norte,1
1,AC,Acre,Norte,1
2,AM,Amazonas,Norte,1
3,RR,Roraima,Norte,1
4,PA,Pará,Norte,1
5,AP,Amapá,Norte,1
6,TO,Tocantins,Norte,1
7,MA,Maranhão,Nordeste,2
8,PI,Piauí,Nordeste,2
9,CE,Ceará,Nordeste,2


In [5]:
 """
    Função que consome a API de Tabelas Fipe e retorna um DataFrame com as tabelas disponíveis.

    O DataFrame resultante contém as colunas:
    - Código da Tabela: Código numérico identificador da tabela Fipe
    - Mês/Ano: Período da tabela Fipe no formato Mês/Ano

    Retorna:
        pd.DataFrame: DataFrame com informações das tabelas Fipe.
        None: Se houver erro na requisição à API.
    """

def tabelas_fipe():
    url = 'https://brasilapi.com.br/api/fipe/tabelas/v1'
    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        tabelas_fipe_df = pd.DataFrame(data_json)
        
        # Tratamento dos dados
        # Renomeando colunas
        tabelas_fipe_df = tabelas_fipe_df.rename(columns={
            'codigo': 'Código da Tabela',
            'mes': 'Mês/Ano'
        })
        
        # Convertendo 'Código da Tabela' para numérico, caso necessário
        tabelas_fipe_df['Código da Tabela'] = pd.to_numeric(tabelas_fipe_df['Código da Tabela'], errors='coerce')
        
        return tabelas_fipe_df
    else:
        print("Erro ao acessar a API de Tabelas Fipe")
        return None

tabelas_fipe()


,Código da Tabela,Mês/Ano
0,314,outubro/2024
1,313,setembro/2024
2,312,agosto/2024
3,311,julho/2024
4,310,junho/2024
...,...,...
281,8,janeiro/2006
282,7,agosto/2005
283,6,novembro/2005
284,2,outubro/2005


In [6]:
"""
    Função que consome a API de corretoras da CVM e retorna um DataFrame com
    informações sobre as corretoras cadastradas.

    O DataFrame resultante contém as colunas:
    - Código CVM: Código da corretora na Comissão de Valores Mobiliários (CVM)
    - Nome Social: Nome social da corretora
    - Nome Comercial: Nome comercial da corretora
    - CNPJ: Cadastro Nacional da Pessoa Jurídica da corretora

    Colunas não relevantes, como 'situacao', são excluídas.

    Retorna:
        pd.DataFrame: DataFrame com informações das corretoras cadastradas.
        None: Se houver erro na requisição à API.
    """

def corretoras():
    url = 'https://brasilapi.com.br/api/cvm/corretoras/v1'
    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        corretoras_df = pd.DataFrame(data_json)
        
        # Tratamento dos dados
        # Renomeando colunas
        corretoras_df = corretoras_df.rename(columns={
            'codigo_cvm': 'Código CVM',
            'nome_social': 'Nome Social',
            'nome_comercial': 'Nome Comercial',
            'cnpj': 'CNPJ'
        })
        
        # Excluindo colunas
        if 'situacao' in corretoras_df.columns:
            corretoras_df = corretoras_df.drop(columns=['situacao'])  # Exemplo de coluna a ser removida
                
        return corretoras_df
    else:
        print("Erro ao acessar a API de Corretoras")
        return None

corretoras().head(5)

,CNPJ,type,Nome Social,Nome Comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,Código CVM,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL DTVM,CANCELADA,complianceregulatorio@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,1989-12-31,5995252.29,329,1990-06-12,1986-07-08
3,44527444000155,CORRETORAS,ABN AMRO CLEARING CORRETORA DE T-TULOS E VALOR...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",2023-12-31,11122779.75,3794,2022-06-20,2022-06-20
4,33819590000138,CORRETORAS,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,,5095755,20040040,BRASIL,RJ,RIO DE JANEIRO,CENTRO,,"PRAÇA OLAVO BILAC, 28/ SL. 608",2000-12-31,291862.38,1112,2001-04-11,1968-04-19


In [14]:
 """
    Salva um DataFrame em uma tabela de um banco de dados SQLite.

    Esta função grava os dados de um DataFrame em uma tabela do banco de dados,
    substituindo a tabela existente se já houver uma com o mesmo nome.

    Parâmetros:
        df (pd.DataFrame): O DataFrame que será salvo no banco de dados.
        tabela_nome (str): O nome da tabela onde os dados serão armazenados.
        conexao (sqlite3.Connection): A conexão ativa com o banco de dados SQLite.

    O DataFrame é gravado na tabela especificada usando o método `to_sql` do pandas.
    Se a tabela já existir, ela será substituída.

    Retorna:
        None: A função não retorna valor; apenas salva os dados no banco.
    """

def salvar_bd(df, tabela_nome, conexao):
        df.to_sql(tabela_nome, conexao, if_exists='replace', index=False)

conexao = sqlite3.connect('dados_api.db')


df_estados_ibge = estados_ibge()
df_tabela_fipe = tabelas_fipe()
df_corretoras = corretoras()